In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../data/clean/dataset.csv')

In [3]:
df

,name,url,title,lemmes,funny
0,liberation,https://www.liberation.fr/international/ameriq...,"A New York, la mairie veut rendre la Ve Avenue...",new york mairie vouloir ve avenue pieton,0
1,liberation,https://www.liberation.fr/forums/vers-une-resp...,Vers une responsabilité sociale et environneme...,responsabilite social environnemental musee,0
2,liberation,https://www.liberation.fr/international/ameriq...,Présidentielle américaine 2024 : à la frontièr...,presidentielle americain frontiere mexicain es...,0
3,liberation,https://www.liberation.fr/forums/mal-logement-...,Mal-logement : «Les luttes des habitants sont ...,mal-logement lutte habitant revendication ecol...,0
4,liberation,https://www.liberation.fr/culture/musique/waru...,"Warum Joe, punk canal historique",warum joe punk canal historique,0
...,...,...,...,...,...
102080,le_parisien,https://www.leparisien.fr/international/etats-...,« Nous avons désormais une chance de gagner »...,desormais chance gagner apres retrait biden de...,0
102081,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : pourquoi le tournoi ...",jo paris handball tournoi terminer lille paris,0
102082,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : quels sont les pays ...",jo paris handball pays titres palmares tournoi...,0
102083,le_parisien,https://www.leparisien.fr/jo-paris-2024/handba...,"JO Paris 2024, handball : le palmarès complet...",jo paris handball palmares complet equipe fran...,0


In [4]:
print(len(df))
print(len(df[df['funny'] == 0]))
print(len(df[df['funny'] == 0]) / len(df))
print(len(df[df['funny'] == 1]))
print(len(df[df['funny'] == 1]) / len(df))

102085
86795
0.8502228535044326
15290
0.14977714649556742


In [5]:
df['name'].unique()

array(['liberation', 'huffpost', 'ouest_france', 'actu_orange', 'europe1',
       'franceinfotv', 'le_monde', 'la_depeche', 'rfi', 'demotivateur',
       'le_parisien'], dtype=object)

In [6]:
df.groupby('name').size().reset_index(name='count').sort_values(by='count', ascending=False)

,name,count
7,le_parisien,17996
9,ouest_france,16704
5,la_depeche,16440
3,franceinfotv,14142
4,huffpost,10307
6,le_monde,8565
10,rfi,7247
8,liberation,5999
1,demotivateur,2381
2,europe1,1704


In [7]:
for name in df['name'].unique():
    print("{:<15} {:>5}    {:>.5f}".format(name, len(df[df['name'] == name]), len(df[df['name'] == name])/len(df)))





liberation       5999    0.05876
huffpost        10307    0.10096
ouest_france    16704    0.16363
actu_orange       600    0.00588
europe1          1704    0.01669
franceinfotv    14142    0.13853
le_monde         8565    0.08390
la_depeche      16440    0.16104
rfi              7247    0.07099
demotivateur     2381    0.02332
le_parisien     17996    0.17628


In [8]:
for index, row in df.groupby('name').size().reset_index(name='count').sort_values(by='count', ascending=False).iterrows():
    print("{:<15} {:>5}    {:>.5f}".format(row['name'], row['count'], row['count']/len(df)))


le_parisien     17996    0.17628
ouest_france    16704    0.16363
la_depeche      16440    0.16104
franceinfotv    14142    0.13853
huffpost        10307    0.10096
le_monde         8565    0.08390
rfi              7247    0.07099
liberation       5999    0.05876
demotivateur     2381    0.02332
europe1          1704    0.01669
actu_orange       600    0.00588
